<img src="https://datascientest.fr/train/assets/logo_datascientest.png" style="height:150px">

<hr style="border-width:2px;border-color:#75DFC1">
<center><h1>Projet Data Science</h1></center>
<center><h2>Introduction</h2></center>
<hr style="border-width:2px;border-color:#75DFC1">

<blockquote>
Ce module est destiné à montrer le déroulement d'un projet de Data Science et d'illuster les différentes étapes.<br>
Le problème que nous allons essayé de résoudre est un problème d'attrition client (ie de <i>churn</i>): nous voulons savoir si des clients vont revenir ou s'ils vont partir. <br> Cette problèmatique peut avoir plusieurs objectifs: proposer des offres à certains clients qui ne devraient théoriquement pas revenir pour justement leur faire changer d'avis, connaître les raisons qui font que les clients ne reviennent pas ou même, tout simplement, prévoir le chiffre d'affaire sur une période à venir. <br>
<br>
Lorsque l'on définit une telle problématique à résoudre, il est intéressant de se poser la question des données à disposition: à quelles données ai-je accès (en interne ou en open data)? Si mes données ne sont pas suffisantes, comment puis-je les compléter (acheter des données, trouver des jeux libres d'accès sur internet, constituer soi-même des jeux de données supplémentaires avec du <i>scraping</i> par exemple, ...)? Est-ce que je connais ce que les données signifient? Et si non, qui peut me renseigner? Comment ces données ont été constituée? Quand l'ont-elles été?  Est-ce que ces données sont de qualité? 

<br>
<br>
Ici, nous n'allons pas essayer de répondre à ces questions mais elles sont essentielles pour proposer des modèles pertinents tout en connaissant les limites de ce modèle. Le jeu de données utilisé ici est nommé <code>events.csv</code> et contient une liste d'événements sur un site de e-commerce. Il fait partie d'un ensemble plus complexes de données disponible <a href="https://www.kaggle.com/retailrocket/ecommerce-dataset">ici</a>.
</blockquote>

* importer <code>pandas</code> et charger le jeu de données
* afficher les 5 premières lignes    

In [1]:
# Insérer le code ici

In [2]:
# Insérer le code ici

import pandas as pd 

df = pd.read_csv('events.csv')
df.head()

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN


* vérifier le type des données avec la méthode <code>info</code>

In [3]:
# Insérer le code ici

In [4]:
# Insérer le code ici

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2756101 entries, 0 to 2756100
Data columns (total 5 columns):
timestamp        int64
visitorid        int64
event            object
itemid           int64
transactionid    float64
dtypes: float64(1), int64(3), object(1)
memory usage: 105.1+ MB


<hr style="border-width:2px;border-color:#75DFC1">
<center><h2>Description des données</h2></center>
<hr style="border-width:2px;border-color:#75DFC1">

<blockquote>
Les variables contenues dans ce jeu de données sont:
<ul>
    <li><code>timestamp</code>: timestamp représentant la date et l'heure à laquelle a eu lieu l'événement.</li>
    <li><code>visitorid</code>: nombre qui représente un unique visiteur du site acteur de l'événement.</li>
    <li><code>event</code>: nature de l'événement. Peut être de 3 types différents: "view", "addtocart", "transaction" qui correspondent à la vue d'un objet, son ajout au panier ou son achat.</li>
    <li><code>itemid</code>: nombre qui représente un unique produit concerné par l'événement.</li>
    <li><code>transactionid</code>: nombre qui représente une transaction unique</li>
</ul>
    
Une des premières choses à faire est de faire l'inventaire du nombre de produits, du nombre de visiteurs, du nombre d'événements, ...             
</blockquote>

* donner le nombre d'événements, le nombre de visiteurs uniques, le nombre de produits uniques, la date de début de collecte des données et la date de fin de collecte (pour ce dernier point, on peut utiliser le package <code>datetime</code> et en particulier la fonction <code>datetime.fromtimestamp</code>.

In [5]:
# Insérer le code ici

import datetime

In [6]:
# Insérer le code ici

import datetime

n_events = len(df)  # le nombre d'événements correspond au nombre de lignes dans le jeu de données
n_visitors = len(df.visitorid.unique())  # le nombre de visiteurs uniques correspond au nombre de valeurs différentes de visitorid
n_products = len(df.itemid.unique())  # le nombre de produits uniques correspond au nombre de valeurs différentes de visitorid


print(u'nombre d\'événements uniques: {}'.format(n_events))
print(u'nombre de visiteurs uniques: {}'.format(n_visitors))
print(u'nombre de produits uniques: {}'.format(n_products))

start_ts, stop_ts = df.timestamp.min(), df.timestamp.max()  # on prend le min et le max de la colonne timestamp 
start = datetime.datetime.fromtimestamp(start_ts/1000)  # on change le timestamp and objet date /!\ la fonction prend des secondes et ici on a des milisecondes
stop = datetime.datetime.fromtimestamp(stop_ts/1000)  # on change le timestamp and objet date /!\ la fonction prend des secondes et ici on a des milisecondes

print(u'période: {}-{}'.format(start, stop))

nombre d'événements uniques: 2756101
nombre de visiteurs uniques: 1407580
nombre de produits uniques: 235061
période: 2015-05-03 05:00:04.384000-2015-09-18 04:59:47.788000


<blockquote>
Nous avons donc: 
<ul>
    <li> 2 756 101 événements uniques</li>
    <li> 1 407 580 visiteurs uniques </li>
    <li> 235 061 produits uniques </li>
</ul>

Ces événements sont collectés sur une période s'étalant de mai 2015 à septembre 2015.<br>
Nous devons aussi voir quelles sont les données manquantes: 

</blockquote>

* donner le nombre de données manquantes par variables

In [7]:
# Insérer le code ici

In [8]:
# Insérer le code ici

df.isna().sum()

timestamp              0
visitorid              0
event                  0
itemid                 0
transactionid    2733644
dtype: int64

<blockquote>
On constate que la seule variable où il manque des données est la variable <code>transactionid</code>: c'est tout à fait logique car les événements qui ne sont pas une transaction ne peuvent pas avoir de <code>transactionid</code>. Vérifions tout de même que le champs est bien renseigné pour toutes les transactions.
</blockquote>

* vérifier que les événements <code>transaction</code> ont bien un <code>transactionid</code> renseigné à chaque fois. 

In [9]:
# Insérer le code ici

In [10]:
# Insérer le code ici

print(df[df['event'] == 'transaction'].isna().sum())
print(df[df['event'] != 'transaction'].isna().sum())

timestamp        0
visitorid        0
event            0
itemid           0
transactionid    0
dtype: int64
timestamp              0
visitorid              0
event                  0
itemid                 0
transactionid    2733644
dtype: int64


<blockquote>
    Ici nous avons de la chance: le jeu de données est tres propre (il a été probablement nettoyé avant d'être mis sur Kaggle). Cela dit cette étape est très importante: elle nous permet de voir très vite quelles sont les étapes à mettre en place pour pallier à ces données manquantes: 
    <ul>
        <li>supprimer ces lignes au risque de manquer de data ou de perdre une information précieuse</li>
        <li>remplacer ces données par des moyennes, des médianes ou des modes, quitte à trop généraliser</li>
        <li>proposer un modèle plus fin (par exemple avec de l'open data: si j'ai le CSP d'un client, à partir des données de l'INSEE je peux déduire une tranche de salaire)</li>
        <li>proposer un modèle de Machine Learning pour remplacer ces données, ce qui demande du temps et de l'énergie pour des résultats non garantis.</li>
    </ul>
    Aucune de ces options n'est meilleure que les autres: ce sont simplement des choix qui dépendent de beaucoup de facteurs qui entourent ce projet de Data Science: main d'oeuvre, délais, resources, expertise, exigence de résultats, ...
    
Finalement, nous devons vérifier que les modèles de données proposés sont bien ceux que nous avons devant nous. Il faut notamment vérifier que la colonne <code>event</code> ne contient bien que les valeurs avancées plus tôt: une erreur dans la saisie, la collecte, dans le stockage ou tout simplement une mauvaise connaissance des données pourraient nous réserver des surprises. 
</blockquote>
* afficher les différentes modalités de la variable <code>event</code>.

In [11]:
# Insérer le code ici

In [12]:
# Insérer le code ici

print(df.event.unique())

['view' 'addtocart' 'transaction']


<blockquote>

Les informations données plus tôt étaient effectivement les bonnes. On peut déjà déduire plusieurs choses de cette méta-analyse: 
<ul> 
    <li>Nous savons ce à quoi correspond chacune des variables, leurs modalités ou leur format</li>
    <li>Nous n'allons pas avoir de problème de données manquantes, ce qui va nous faire gagner pas mal de temps.</li>
    <li>Le jeu de données est plutôt conséquent: plus de 2,7 millions de lignes: il faudra faire attention à ce facteur et peut être adapté les resources en fonction</li>
    </ul>
    Cette première étape nous permet donc de dresser les contours de notre jeu de données et donc d'accélérer la suite.
    </blockquote>